<a href="https://colab.research.google.com/github/andreiarossi/Genini_Simple_LLM/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Google SDK**

In [1]:
!pip install -q -U google-generativeai

**Imports**

In [57]:
# Import the Python SDK
import pandas as pd
import numpy as np
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

Verify EmbedModels
https://ai.google.dev/gemini-api/docs/embeddings?hl=pt-br

**Ckeking the models versions**

In [3]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)


models/embedding-001
models/text-embedding-004


**Embeding EXEMPLE**

In [6]:
title = 'Título do assunto desejado'
sample_text = ("Conteúdo inicial."
  "\n"
  "Texto completo para leitura:"
  "\n"
  "O texto usado deve ser relacionado ao conteúdo estudado")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [0.024470402, -0.048381735, -0.04074992, -0.003745563, 0.045838032, 0.013582221, -0.0081079155, -0.013161906, -0.018862016, 0.056980733, -0.007910962, 0.021160256, 0.0001250103, -0.009849762, 0.018063258, -0.015170059, 0.030805368, 0.028109385, -0.023156524, -0.010971722, -0.014883397, 0.022299474, -0.0037814921, -0.03137723, -0.0030920599, -0.027632315, 0.012665942, -0.041502226, -0.030197794, 0.00793291, -0.03830562, 0.039077476, -0.029721957, 0.01948549, -0.006431529, -0.02786623, -0.0155216325, 0.026219176, -0.00077274133, -0.009484445, -0.014263129, -0.0068810205, -0.020167245, 0.023814367, 0.0033748387, -0.009531246, -0.026986655, 0.031734295, 0.009107547, -0.042028733, -0.0111794835, -0.006861845, 0.08797621, -0.006091629, 0.031144196, -0.040683765, 0.059492923, -0.00086666556, -0.0054108542, -0.0026610813, -0.030533075, -0.004701019, 0.03515564, 0.032038912, 0.001460381, -0.048519824, -0.03182654, -0.00059229444, 0.031010237, -0.01521818, -0.00553469, -0.034318235

**Starting the LLM structure.**

In [88]:
document1 = {
    "Título": "A revolução elétrica está nas ruas",
    "Conteúdo": "Preparado para as ruas brasileiras, o Renault Kwid E-Tech 100% elétrico está revolucionando a mobilidade do país. Robustez de SUV com altura do solo de 16,6cm, 185 km* de autonomia com uma carga, e pode ser recarregado em qualquer lugar: tomadas, wallbox ou em mais de 3.000 pontos espalhados pelo país. Você ainda conta com a confiabilidade da maior e mais preparada rede de concessionárias de carros 100% elétricos. Encontre a mais próxima, faça um test drive e descubra o melhor da experiência elétrica."
}
document2 = {
    "Título": "Sobre a BYD",
    "Conteúdo": "Fundada em 1995, a BYD é uma empresa líder em tecnologia dedicada a alavancar inovações para uma vida melhor. Com mais de 28 anos de experiência, a BYD se estabeleceu como líder do setor de eletrônicos, automotivo, energia renovável e transporte ferroviário. Como líder global com mais de 30 parques industriais em 6 continentes, as soluções de emissão zero da BYD, focadas na geração e armazenamento de energia, são expansivas e amplamente aplicáveis."
}
document3 = {
    "Título": "Seu primeiro elétrico!",
    "Conteúdo": "O JAC E-JS1 possui baterias de fosfato de ferro-lítio, Provido de sistema de regeneração de carga, o modelo urbano é extremamente econômico: 10 kWh a cada 100 km. E-JS1 possui um excepcional rendimento, à medida que ele é leve. São somente 1.180 kg de peso em ordem de marcha. Por este motivo, o motor de 150 Nm (15,3 kgfm) de torque entrega um desempenho empolgante para um modelo urbano. São 62 cv de potência e ele vai de  0 a 100 km/h em 10,7 segundos!"
}

documents = [document1, document2, document3]

**Convert to datasheet structure.**

In [89]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,A revolução elétrica está nas ruas,"Preparado para as ruas brasileiras, o Renault ..."
1,Sobre a BYD,"Fundada em 1995, a BYD é uma empresa líder em ..."
2,Seu primeiro elétrico!,O JAC E-JS1 possui baterias de fosfato de ferr...


**Using the model.**

In [90]:
model = "models/embedding-001"


**Defining functions to read embeddings.**

In [91]:
def embed_fn(tittle, text):
  return genai.embed_content(model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

**Creating a new collum to receive embedding code**

In [92]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,A revolução elétrica está nas ruas,"Preparado para as ruas brasileiras, o Renault ...","[0.05601931, -0.0038169993, -0.04414298, -0.01..."
1,Sobre a BYD,"Fundada em 1995, a BYD é uma empresa líder em ...","[0.0073059625, -0.025836313, -0.050422512, 0.0..."
2,Seu primeiro elétrico!,O JAC E-JS1 possui baterias de fosfato de ferr...,"[0.028698934, -0.046775434, -0.013149687, -0.0..."


**Distance - to compare**

In [93]:
def build_and_bring_find(consult, base, model):
  embedding_find = genai.embed_content(model=model,
                                       content=consult,
                                       task_type="RETRIEVAL_QUERY")["embedding"]
  prod_scalar = np.dot(np.stack(df['Embeddings']), embedding_find)
  indice = np.argmax(prod_scalar)
  return df.iloc[indice]["Conteudo"]



**Make the inquiry.**



In [94]:
consult = "Criada na década de 90?"

retornar = build_and_bring_find(consult, df, model)
print(retornar)

Fundada em 1995, a BYD é uma empresa líder em tecnologia dedicada a alavancar inovações para uma vida melhor. Com mais de 28 anos de experiência, a BYD se estabeleceu como líder do setor de eletrônicos, automotivo, energia renovável e transporte ferroviário. Como líder global com mais de 30 parques industriais em 6 continentes, as soluções de emissão zero da BYD, focadas na geração e armazenamento de energia, são expansivas e amplamente aplicáveis.
